In [3]:
import pandas as pd
import os
import json
import random
from sklearn.model_selection import train_test_split
import re

### Claim Reasoning (Without question answer pairs)

In [6]:
def process_training_data(input_path, train_output_path, test_output_path, test_size=0.2):
    # Load the processed JSON data
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_claims = []

    for article in data:
        article_id = article["article_id"]

        # Process regular claims
        for claim, reasoning in zip(article["claims"], article["reasonings"]):
            # Clean double spaces
            claim = re.sub(r'\s{2,}', ' ', claim)
            reasoning = re.sub(r'\s{2,}', ' ', reasoning)
            
            reasoning

            # Extract first sentence of reasoning
            reasoning_split = reasoning.split('.', 1)
            reasoning_first_sentence = reasoning_split[0].strip()
            reasoning_remainder = reasoning_split[1].strip() if len(reasoning_split) > 1 else ""

            # Determine agreement type
            if "evet" in reasoning_first_sentence.lower():
                agreement = "Evet"
                reasoning = reasoning_remainder
            elif "hayır" in reasoning_first_sentence.lower():
                agreement = "Hayır"
                reasoning = reasoning_remainder
            else:
                agreement = "Unknown"

            # Add regular claim to flattened list
            flattened_claims.append({
                "article_id": article_id,
                "claim_id": len(flattened_claims) + 1,
                "claim": claim,
                "agreement": agreement,
                "reasoning": reasoning
            })

        # Process contradicting claims
        for contradicting_claim, contradicting_reasoning in zip(article["contradicting_claims"], article["contradicting_reasonings"]):
            # Clean double spaces
            contradicting_claim = re.sub(r'\s{2,}', ' ', contradicting_claim)
            contradicting_reasoning = re.sub(r'\s{2,}', ' ', contradicting_reasoning)

            # Extract first sentence of contradicting reasoning
            contradicting_reasoning_split = contradicting_reasoning.split('.', 1)
            contradicting_reasoning_first_sentence = contradicting_reasoning_split[0].strip()
            contradicting_reasoning_remainder = contradicting_reasoning_split[1].strip() if len(contradicting_reasoning_split) > 1 else ""

            # Determine agreement type
            if "evet" in contradicting_reasoning_first_sentence.lower():
                agreement = "Evet"
                contradicting_reasoning = contradicting_reasoning_remainder
            elif "hayır" in contradicting_reasoning_first_sentence.lower():
                agreement = "Hayır"
                contradicting_reasoning = contradicting_reasoning_remainder
            else:
                agreement = "Unknown"

            # Add contradicting claim to flattened list
            flattened_claims.append({
                "article_id": article_id,
                "claim_id": len(flattened_claims) + 1,
                "claim": contradicting_claim,
                "agreement": agreement,
                "reasoning": contradicting_reasoning
            })

    # Prepare data for stratified splitting
    agreements = [claim["agreement"] for claim in flattened_claims]

    # Stratified train-test split
    train_data, test_data = train_test_split(flattened_claims, test_size=test_size, stratify=agreements, random_state=42)

    # Save the training data
    os.makedirs(os.path.dirname(train_output_path), exist_ok=True)
    with open(train_output_path, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)

    # Save the testing data
    os.makedirs(os.path.dirname(test_output_path), exist_ok=True)
    with open(test_output_path, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=4)

    print(f"Training data saved to {train_output_path}")
    print(f"Testing data saved to {test_output_path}")

In [7]:
columnists = ["ahmethakan", "abdulkadirselvi"]

for columnist in columnists:
    input_json = f"../../../columnist_data/claim_reasoning/{columnist}.json"
    train_output_json = f"../../../finetune_data/claim_reasoning/{columnist}/{columnist}_train.json"
    test_output_json = f"../../../finetune_data/claim_reasoning/{columnist}/{columnist}_test.json"
    process_training_data(input_json, train_output_json, test_output_json)

Training data saved to ../../../finetune_data/claim_reasoning/ahmethakan/ahmethakan_train.json
Testing data saved to ../../../finetune_data/claim_reasoning/ahmethakan/ahmethakan_test.json
Training data saved to ../../../finetune_data/claim_reasoning/abdulkadirselvi/abdulkadirselvi_train.json
Testing data saved to ../../../finetune_data/claim_reasoning/abdulkadirselvi/abdulkadirselvi_test.json


### Claim Questions (With question answer pairs)

In [8]:
def process_training_data(input_path, train_output_path, test_output_path, test_size=0.2):
    # Load the processed JSON data
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_claims = []

    for article in data:
        article_id = article["article_id"]

        # Process regular claims
        for claim, reasoning, question in zip(article["claims"], article["reasonings"], article["questions"]):
            # Clean double spaces
            claim = re.sub(r'\s{2,}', ' ', claim)
            reasoning = re.sub(r'\s{2,}', ' ', reasoning)
            

            # Extract first sentence of reasoning
            reasoning_split = reasoning.split('.', 1)
            reasoning_first_sentence = reasoning_split[0].strip()
            reasoning_remainder = reasoning_split[1].strip() if len(reasoning_split) > 1 else ""

            # Determine agreement type
            if "evet" in reasoning_first_sentence.lower():
                agreement = "Evet"
                reasoning = reasoning_remainder
            elif "hayır" in reasoning_first_sentence.lower():
                agreement = "Hayır"
                reasoning = reasoning_remainder
            else:
                agreement = "Unknown"

            # Add regular claim to flattened list
            flattened_claims.append({
                "article_id": article_id,
                "claim_id": len(flattened_claims) + 1,
                "claim": claim,
                "question": question,
                "agreement": agreement,
                "reasoning": reasoning
            })

        # Process contradicting claims
        for contradicting_claim, contradicting_reasoning, question in zip(article["contradicting_claims"], article["contradicting_reasonings"], article["questions"]):
            # Clean double spaces
            contradicting_claim = re.sub(r'\s{2,}', ' ', contradicting_claim)
            contradicting_reasoning = re.sub(r'\s{2,}', ' ', contradicting_reasoning)

            # Extract first sentence of contradicting reasoning
            contradicting_reasoning_split = contradicting_reasoning.split('.', 1)
            contradicting_reasoning_first_sentence = contradicting_reasoning_split[0].strip()
            contradicting_reasoning_remainder = contradicting_reasoning_split[1].strip() if len(contradicting_reasoning_split) > 1 else ""

            # Determine agreement type
            if "evet" in contradicting_reasoning_first_sentence.lower():
                agreement = "Evet"
                contradicting_reasoning = contradicting_reasoning_remainder
            elif "hayır" in contradicting_reasoning_first_sentence.lower():
                agreement = "Hayır"
                contradicting_reasoning = contradicting_reasoning_remainder
            else:
                agreement = "Unknown"

            # Add contradicting claim to flattened list
            flattened_claims.append({
                "article_id": article_id,
                "claim_id": len(flattened_claims) + 1,
                "claim": contradicting_claim,
                "question": question,
                "agreement": agreement,
                "reasoning": contradicting_reasoning
            })

    # Prepare data for stratified splitting
    agreements = [claim["agreement"] for claim in flattened_claims]

    # Stratified train-test split
    train_data, test_data = train_test_split(flattened_claims, test_size=test_size, stratify=agreements, random_state=42)

    # Save the training data
    os.makedirs(os.path.dirname(train_output_path), exist_ok=True)
    with open(train_output_path, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)

    # Save the testing data
    os.makedirs(os.path.dirname(test_output_path), exist_ok=True)
    with open(test_output_path, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=4)

    print(f"Training data saved to {train_output_path}")
    print(f"Testing data saved to {test_output_path}")

In [9]:
columnists = ["ahmethakan", "abdulkadirselvi"]

for columnist in columnists:
    input_json = f"../../../columnist_data/claim_questions/{columnist}.json"
    train_output_json = f"../../../finetune_data/claim_questions/{columnist}/{columnist}_train.json"
    test_output_json = f"../../../finetune_data/claim_questions/{columnist}/{columnist}_test.json"
    process_training_data(input_json, train_output_json, test_output_json)

Training data saved to ../../../finetune_data/claim_questions/ahmethakan/ahmethakan_train.json
Testing data saved to ../../../finetune_data/claim_questions/ahmethakan/ahmethakan_test.json
Training data saved to ../../../finetune_data/claim_questions/abdulkadirselvi/abdulkadirselvi_train.json
Testing data saved to ../../../finetune_data/claim_questions/abdulkadirselvi/abdulkadirselvi_test.json
